In [76]:
import pandas as pd
data = pd.read_parquet('src/data/data')

In [77]:
data['conversion'].unique()

array([0, 1])

In [78]:
data.head()

,boosted,full_name,free_shipping,conversion,is_pdp,logistic_type,original_price,platform,price,print_position,print_server_timestamp,title,device_type,taxonomy_level,taxonomy_level1,taxonomy_level2
0,False,Celulares y Teléfonos -> Celulares y Smartphones,1,0,0,cross_docking,29999,web,20999,1717,2020-03-06T00:19:30.735-0400,Motorola One Action 128 Gb Denim Blue 4 Gb Ram,desktop,3,Celulares y Teléfonos,Celulares y Smartphones
1,False,Deportes y Fitness -> Zapatillas,1,0,0,drop_off,6700,mobile,3999,311,2020-04-01T21:20:11.738-0400,Zapatillas Hovr Sonic 2 Under Armour,android,3,Deportes y Fitness,Zapatillas
2,False,"Ropa y Accesorios -> Pantalones, Jeans y Joggings",0,0,0,fulfillment,1490,mobile,1341,1686,2020-04-08T18:38:48.360-0400,Pantalon Mujer Vestir Elastizado Tiro Alto Tal...,android,3,Ropa y Accesorios,"Pantalones, Jeans y Joggings"
3,False,Juegos y Juguetes -> Vehículos de Juguete -> V...,1,0,0,fulfillment,3498,web,3007,12,2020-04-25T22:01:19.829-0400,Trencity Pista N°3 - Kit Turbo Elevación + Pu...,mobile,5,Juegos y Juguetes,Vehículos de Juguete
4,False,Computación -> Componentes de PC -> Memorias RAM,0,0,0,cross_docking,1248,mobile,998,511,2020-03-10T13:20:56.633-0400,Memoria Ram Ddr2 2gb 800mhz Samsung - Intel Y Amd,android,4,Computación,Componentes de PC


In [79]:
data['boosted'] = data['boosted'].apply(lambda x: int(x))
y = pd.get_dummies(data.platform, prefix='platform')
x = pd.get_dummies(data.device_type, prefix='device_type')

In [80]:
data = pd.concat([data,y,x], axis=1)
df = data.query("taxonomy_level1=='Ropa y Accesorios '")
df

,boosted,full_name,free_shipping,conversion,is_pdp,logistic_type,original_price,platform,price,print_position,...,device_type,taxonomy_level,taxonomy_level1,taxonomy_level2,platform_mobile,platform_web,device_type_android,device_type_desktop,device_type_ios,device_type_mobile
2,0,"Ropa y Accesorios -> Pantalones, Jeans y Joggings",0,0,0,fulfillment,1490,mobile,1341,1686,...,android,3,Ropa y Accesorios,"Pantalones, Jeans y Joggings",1,0,1,0,0,0
17,0,Ropa y Accesorios -> Calzado -> Zapatillas,1,0,0,cross_docking,1221,mobile,1038,103,...,android,4,Ropa y Accesorios,Calzado,1,0,1,0,0,0
18,0,Ropa y Accesorios -> Calzado -> Zapatillas,1,0,0,cross_docking,4789,mobile,3689,7718,...,ios,4,Ropa y Accesorios,Calzado,1,0,0,0,1,0
25,0,Ropa y Accesorios -> Calzado -> Zapatillas,0,0,0,cross_docking,900,mobile,801,1704,...,android,4,Ropa y Accesorios,Calzado,1,0,1,0,0,0
34,0,Ropa y Accesorios -> Camisas,0,0,0,cross_docking,1600,mobile,1360,80,...,android,3,Ropa y Accesorios,Camisas,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19179,0,Ropa y Accesorios -> Accesorios de Moda -> Gor...,0,0,0,cross_docking,699,mobile,594,474,...,android,4,Ropa y Accesorios,Accesorios de Moda,1,0,1,0,0,0
19186,0,Ropa y Accesorios -> Calzado -> Zapatillas,0,0,0,drop_off,3000,mobile,989,312,...,android,4,Ropa y Accesorios,Calzado,1,0,1,0,0,0
19198,0,Ropa y Accesorios -> Calzado -> Zapatillas,1,0,0,drop_off,4490,mobile,2999,76,...,android,4,Ropa y Accesorios,Calzado,1,0,1,0,0,0
19199,0,Ropa y Accesorios -> Calzado -> Zapatillas,0,0,0,cross_docking,805,mobile,724,1340,...,android,4,Ropa y Accesorios,Calzado,1,0,1,0,0,0


In [81]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [82]:
X = df[['boosted','free_shipping','is_pdp','original_price','price','print_position','platform_mobile','device_type_desktop','device_type_ios','device_type_mobile']]
y = df['conversion']

print(y)

2        0
17       0
18       0
25       0
34       0
        ..
19179    0
19186    0
19198    0
19199    0
19210    0
Name: conversion, Length: 30633, dtype: int64


In [83]:
clf = LogisticRegression(random_state=0).fit(X, y)
clf.score(X, y)

0.9393791009695427

In [84]:
import statsmodels.api as sm
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             conversion   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     58.40
Date:                Sun, 20 Nov 2022   Prob (F-statistic):           1.15e-83
Time:                        17:45:02   Log-Likelihood:                 628.52
No. Observations:               30633   AIC:                            -1241.
Df Residuals:                   30625   BIC:                            -1174.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
boosted                      0    

/home/jhkratos/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1934: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [85]:
data['taxonomy_level1'].value_counts().reset_index()

,index,taxonomy_level1
0,"Hogar, Muebles y Jardín",34035
1,Ropa y Accesorios,30633
2,Herramientas y Construcción,17673
3,Deportes y Fitness,17305
4,Belleza y Cuidado Personal,15079
5,"Electrónica, Audio y Video",14753
6,Celulares y Teléfonos,13712
7,Electrodomésticos y Aires Ac.,8554
8,Juegos y Juguetes,7605
9,Bebés,6553
